In [1]:
import numpy as np
import scipy

def GPM_solve(A_L, A_U):
    n = A_L.shape[0]
    A_ub1 = np.hstack((np.ones((n, n)) - np.eye(n), np.zeros((n, n)) + np.eye(n)))
    A_lb1 = np.hstack((np.zeros((n, n)) + np.eye(n), np.ones((n, n)) - np.eye(n)))
    A_lb2 = np.hstack([-np.eye(n), np.eye(n)])
    
    A_lb3 = np.hstack([-np.eye(n)*(n-1), A_L - np.eye(n)])
    A_lb4 = np.hstack([A_U - np.eye(n), -np.eye(n)*(n-1)])
    
    A_ub_full = np.vstack([A_ub1, -A_lb1, -A_lb2, -A_lb3, -A_lb4])
    b_ub_full = np.hstack([np.ones(n), -np.ones(n), -np.zeros(n), np.zeros(n), np.zeros(n)])
    
    c1 = np.ones(n).dot(A_U - n * np.eye(n))
    c2 = np.ones(n).dot(A_L - n * np.eye(n))
    c_full = np.hstack([c1, c2])

    solution = scipy.optimize.linprog(
        c=c_full,
        A_ub=A_ub_full,
        b_ub=b_ub_full,
    )
    print("Linear programming problem solution:")
    print(solution)
    print(f"J*={solution.fun}")
    w_l, w_u = np.array_split(solution.x, 2)
    return w_l, w_u

def rank(w_l, w_u):
    n = w_l.shape[0]
    P = np.empty((n, n))
    for i in range(n):
        for j in range(n):
            a_l, a_u = w_l[i], w_u[i]
            b_l, b_u = w_l[j], w_u[j]
            P[i, j] = (max(a_u - b_l, 0) - max(a_l - b_u, 0)) / ((a_u - b_l) - (a_l - b_u))

    ranking = P.sum(1)[::-1].argsort()
    return list(ranking)

def power_method(D):
    x_prev = np.random.uniform(size=D.shape[0])
    x_prev /= np.linalg.norm(x_prev)
    
    while True:
        x_cur = D.dot(x_prev)
        lam_cur = np.linalg.norm(x_cur)
        x_cur /= np.linalg.norm(x_cur)

        if np.linalg.norm(x_cur - x_prev) < 1e-5:
            return x_cur / x_cur.sum(), lam_cur

        x_prev = x_cur

def compute_CR(lam_max, N):
    CI = (lam_max - N) / (N-1)
    MRCI = [0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.4, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59]
    THRESH = [0, 0, 0.05, 0.08, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
    CR = CI / MRCI[N-1]
    print(f"CR={CR:.3f}")
    thresh = THRESH[N-1]
    if CR > thresh:
        print(f"CR > {thresh} - МПП неузгоджена")
    elif CR > 0:
        print(f"CR <= {thresh} - МПП допустимо узгоджена")
    else:
        print(f"CR = 0.0 - МПП узгоджена")
    return CR


In [2]:
A_L = np.array([
    [1.0,   1.0,   3.0,   5.0,   5.0],
    [1/3.0, 1.0,   1.0,   1.0,   1.0],
    [1/5.0, 1/4.0, 1.0,   1/5.0, 2.0],
    [1/7.0, 1/5.0, 1/5.0, 1.0,   1.0],
    [1/9.0, 1/4.0, 1/4.0, 1/2.0, 1.0],
])

A_U = np.array([
    [1.0,   3.0,   5.0,   7.0,   9.0],
    [1.0,   1.0,   4.0,   5.0,   4.0],
    [1/3.0, 1.0,   1.0,   5.0,   4.0],
    [1/5.0, 1.0,   5.0,   1.0,   2.0],
    [1/5.0, 1.0,   1/2.0, 1.0,   1.0],
])

In [3]:
w_l, w_u = GPM_solve(A_L, A_U)
ranking = rank(w_l, w_u)
for i_rank, rank in enumerate(ranking):
    print(f"Top #{i_rank} alternative: #{rank} wL={w_l[rank]:.3f} wU={w_u[rank]:.3f}")

print("Solving a non-fuzzy relaxed problem")
A_relaxed = np.sqrt(A_L * A_U)
w_EM, lam_EM = power_method(A_relaxed)
compute_CR(lam_EM, w_EM.shape[0])
print(f"Top eigenvalue: {lam_EM:.3f}")
print(f"Corresponding eigenvec: {w_EM}")
print(f"Ranking: {w_EM.argsort()[::-1]}")

Linear programming problem solution:
        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.4441825057005149
              x: [ 4.527e-01  1.396e-01  8.175e-02  5.907e-02  6.327e-02
                   4.527e-01  3.320e-01  2.097e-01  1.347e-01  6.327e-02]
            nit: 16
          lower:  residual: [ 4.527e-01  1.396e-01  8.175e-02  5.907e-02
                              6.327e-02  4.527e-01  3.320e-01  2.097e-01
                              1.347e-01  6.327e-02]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.0